In [8]:
import mne  # Import MNE package
import matplotlib.pyplot as plt  # Import Matplotlib for plotting
import numpy as np  # Import NumPy for numerical operations
import pandas as pd  # Import Pandas for data manipulation
import seaborn as sns  # Import Seaborn for statistical plotting
import statsmodels.formula.api as smf  # Import StatsModels for GLM
from mne.preprocessing.nirs import beer_lambert_law, optical_density  # Import NIRS processing functions

# Import MNE-BIDS for handling BIDS formatted data
from mne_bids import BIDSPath, get_entity_vals, read_raw_bids

# Import MNE-NIRS processing functions for channel handling
from mne_nirs.channels import get_long_channels, get_short_channels, picks_pair_to_idx
from mne_nirs.datasets import fnirs_motor_group
from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.io.fold import fold_channel_specificity

# Import MNE-NIRS statistics and visualization functions
from mne_nirs.statistics import run_glm, statsmodels_to_results
from mne_nirs.visualisation import plot_glm_group_topo, plot_glm_surface_projection


Set the working directory

In [9]:
import os
os.chdir("/Volumes/dep/Allied Health Professions/NIB Team/Project Folders/DT and PD/fnirs/by_task/dt_simple-bids")

# Print the current working directory to verify it
print(os.getcwd())

/Volumes/dep/Allied Health Professions/NIB Team/Project Folders/DT and PD/fnirs/by_task/dt_simple-bids


Set the path where the raw data is stored (example BIDS dataset)

In [10]:
from pathlib import Path

root = Path('/Volumes/dep/Allied Health Professions/NIB Team/Project Folders/DT and PD/fnirs/by_task/dt_simple-bids')
print(root) 

/Volumes/dep/Allied Health Professions/NIB Team/Project Folders/DT and PD/fnirs/by_task/dt_simple-bids


Create a BIDSPathobject

In [11]:
dataset = BIDSPath(
    root=root, task="SimpleDT", datatype="nirs", suffix="nirs", extension=".snirf"
)

print(dataset.directory)

/Volumes/dep/Allied Health Professions/NIB Team/Project Folders/DT and PD/fnirs/by_task/dt_simple-bids/nirs


Query available entities like subjects in the dataset

In [12]:
subjects = get_entity_vals(root, "subject")  # Fix here by using 'dataset' instead of 'roExecution'
print(subjects)

['pd005', 'pd006', 'pd007', 'pd009', 'pd011', 'pd013', 'pd014', 'pd015', 'pd016', 'pd017', 'pd018', 'pd019', 'pd020', 'pd021', 'pd022', 'pd023', 'pd024', 'pd026', 'pd027', 'pd028', 'pd029', 'pd030', 'pd031', 'pd032', 'pd033', 'pd034', 'pd035', 'pd036', 'pd037', 'pd038', 'pd039', 'pd040', 'pd041', 'pd046']


Define function for individual analysis of a subject's data

In [13]:
def individual_analysis(bids_path, ID):
    raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)
    # Delete annotation labeled 15, as these just signify the experiment start and end.
    raw_intensity.annotations.delete(raw_intensity.annotations.description == "15.0")
    # sanitize event names
    raw_intensity.annotations.description[:] = [
        d.replace("/", "_") for d in raw_intensity.annotations.description
    ]

    # Convert signal to optical density (OD) and then to haemoglobin (HbO/HbR)
    raw_od = optical_density(raw_intensity)
    raw_haemo = beer_lambert_law(raw_od, ppf=0.1)  # ppf = path length (e.g., 0.1 cm)
    raw_haemo.resample(0.3)  # Resample the data to 0.3 Hz (for GLM analysis)

    # Extract short channels (to be used as a GLM regressor)
    sht_chans = get_short_channels(raw_haemo)
    raw_haemo = get_long_channels(raw_haemo)  # Extract long channels for analysis

    # Create a first-level design matrix for GLM analysis (stimulus duration is 5 seconds)
    design_matrix = make_first_level_design_matrix(raw_haemo, stim_dur=5.0)

    # Add mean signal from short channels (HbO and HbR) to design matrix
    design_matrix["ShortHbO"] = np.mean(
        sht_chans.copy().pick(picks="hbo").get_data(), axis=0
    )
    design_matrix["ShortHbR"] = np.mean(
        sht_chans.copy().pick(picks="hbr").get_data(), axis=0
    )
    #### design_matrix["acc"] = acc_data

    # Run the GLM analysis using the design matrix
    glm_est = run_glm(raw_haemo, design_matrix)

    # Define regions of interest (ROIs) by listing channel pairs
    PFC = [[1, 1], [1, 3], [2, 1], [2, 2], [2, 5], [3, 2], [3, 3], [4, 3], [5, 1], [5, 6], [6, 5], [6, 6]]
    PMC = [[3, 4], [6, 7], [7, 7], [8, 7], [8, 6], [9, 4], [10, 4]]
    SMA = [[2, 8], [7, 5], [7, 8], [9, 2], [9, 8]]
    M = [[7, 13], [9, 9], [11, 8], [11, 9], [11, 13]]
    SA = [[11, 12], [14, 12], [14, 13], [15, 9], [15, 12], [16, 12]]
    
    # Generate indices for each ROI based on the channel pairs
    groups = dict(
        PFC_region=picks_pair_to_idx(raw_haemo, PFC, on_missing="ignore"),
        PMC_region=picks_pair_to_idx(raw_haemo, PMC, on_missing="ignore"),
        SMA_region=picks_pair_to_idx(raw_haemo, SMA, on_missing="ignore"),
        M1_region=picks_pair_to_idx(raw_haemo, M, on_missing="ignore"),
        SA_region=picks_pair_to_idx(raw_haemo, SA, on_missing="ignore")
    )

    # Extract channel metrics from the GLM results
    cha = glm_est.to_dataframe()

    # Compute ROI results from the channel data
    roi = glm_est.to_dataframe_region_of_interest(
        groups, design_matrix.columns, demographic_info=True
    )

    # Define the contrast for 2 different tasks (e.g., "ST Walk" vs "OT")
    contrast_matrix = np.eye(design_matrix.shape[1])
    basic_conts = dict(
        [(column, contrast_matrix[i]) for i, column in enumerate(design_matrix.columns)]
    )
    contrast_LvR = basic_conts["ST Walk"] - basic_conts["OT"]

    # Compute the defined contrast
    contrast = glm_est.compute_contrast(contrast_LvR)
    con = contrast.to_dataframe()

    # Add the participant ID to the dataframes for better identification
    roi["ID"] = cha["ID"] = con["ID"] = ID

    # Convert the results to micromolar (uM) for better plotting
    cha["theta"] = [t * 1.0e6 for t in cha["theta"]]
    roi["theta"] = [t * 1.0e6 for t in roi["theta"]]
    con["effect"] = [t * 1.0e6 for t in con["effect"]]

    # Return the results for further analysis or plotting
    return raw_haemo, roi, cha, con

Run analysis on all participants

In [14]:
df_roi = pd.DataFrame()  # To store region of interest results
df_cha = pd.DataFrame()  # To store channel level results
df_con = pd.DataFrame()  # To store channel level contrast results

for sub in subjects:
    # Create path to file based on experiment info
    bids_path = dataset.update(subject=sub)  # This should be used correctly to get the path

    # Analyse data and return both ROI and channel results
    raw_haemo, roi, channel, con = individual_analysis(bids_path, sub)

    # Append individual results to all participants
    df_roi = pd.concat([df_roi, roi], ignore_index=True)
    df_cha = pd.concat([df_cha, channel], ignore_index=True)
    df_con = pd.concat([df_con, con], ignore_index=True)

# Print final results or save them to CSV files for later analysis
print(df_roi.head())
print(df_cha.head())
print(df_con.head())


Reading 0 ... 3626  =      0.000 ...   712.872 secs...


/var/folders/jj/m5hzfl052j3_qxn15v5jc69w0000gp/T/ipykernel_56581/3400737577.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)


Reading 0 ... 3948  =      0.000 ...   776.177 secs...


/var/folders/jj/m5hzfl052j3_qxn15v5jc69w0000gp/T/ipykernel_56581/3400737577.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)


Reading 0 ... 3908  =      0.000 ...   768.313 secs...


...

Reading 0 ... 3829  =      0.000 ...   752.781 secs...
Reading 0 ... 3912  =      0.000 ...   769.099 secs...
Reading 0 ... 3488  =      0.000 ...   685.741 secs...


/var/folders/jj/m5hzfl052j3_qxn15v5jc69w0000gp/T/ipykernel_56581/3400737577.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)


Reading 0 ... 3253  =      0.000 ...   639.540 secs...


/var/folders/jj/m5hzfl052j3_qxn15v5jc69w0000gp/T/ipykernel_56581/3400737577.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)


Reading 0 ... 3816  =      0.000 ...   750.226 secs...
Reading 0 ... 3720  =      0.000 ...   731.352 secs...
          ROI Condition Chroma     theta         se         t  dfe         p  \
0  PFC_region         8    hbo  8.896952  10.047613  0.885479   25  0.384336   
1  PFC_region         8    hbr  0.407797   3.590580  0.113574   25  0.910482   
2  PMC_region         8    hbo  8.418641  10.890534  0.773024   25  0.446756   
3  PMC_region         8    hbr  0.062424   4.101986  0.015218   25  0.987979   
4  SMA_region         8    hbo  5.771213   6.546942  0.881513   25  0.386437   

                 Weighted      Sex     ID  
0  Inverse standard error  unknown  pd005  
1  Inverse standard error  unknown  pd005  
2  Inverse standard error  unknown  pd005  
3  Inverse standard error  unknown  pd005  
4  Inverse standard error  unknown  pd005  
variable         Condition    df           mse   p_value        se         t  \
0                        8  25.0  9.475563e-11  0.712506  0.00000